# Clonamos el repositorio para obtener los dataSet





In [ ]:
!git clone https://github.com/joanby/ia-course.git

fatal: destination path 'ia-course' already exists and is not an empty directory.


# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

#Google colab tools

In [1]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

# Instalar dependencias de Renderizado, tarda alrededor de 45 segundos


In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install python-opengl -y > /dev/null 2>&1
!apt install xvfb -y --fix-missing > /dev/null 2>&1
!apt-get install ffmpeg > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!apt-get install pyglet > /dev/null 2>&1


# Instalar OpenAi Gym

In [3]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install piglet > /dev/null 2>&1
!pip install 'gym[box2d]' > /dev/null 2>&1
#por si quieres algun environment en concreto
#!pip install atari_py > /dev/null 2>&1
#!pip install gym[atari] > /dev/null 2>&1

# Todos los imports necesarios en google colab y helpers para poder visualizar OpenAi

In [4]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

# Activamos una vista, seria como crear un plot de una grafica en python

In [5]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900)) #Puedes modificar el high and width de la pantalla
display.start()

# Este código crea una pantalla virtual para dibujar imágenes del juego. 
## Si se ejecuta localmente, simplemente ignóralo

In [6]:
import os
if type(os.environ.get('DISPLAY')) is not str or \
        len(os.environ.get('DISPLAY')) == 0:
    !bash ../xvfb start
    %env DISPLAY=:1

# Funciones de utilidad para permitir la grabación de video del ambiente del gimnasio y su visualización
## Para habilitar la visualizacion por pantalla , tan solo haz "**environment = wrap_env(environment)**", por ejemplo: **environment = wrap_env(gym.make("MountainCar-v0"))**

In [7]:
import io
import glob
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pathlib import Path

def show_one_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)

        content = ipythondisplay.display(HTML(data='''
        <video alt="test" autoplay loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        '''.format(encoded.decode('ascii'))))
    else: 
        print("Couldn't find video")

def show_videos(video_path='video', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

def wrap_env(env):
    env = gym.wrappers.Monitor(env, './video', force=True)
    return env

# Nuestro Script

In [8]:
environment = wrap_env(gym.make("MountainCar-v0"))

- ### EPISILON_MIN : vamos aprendiendo, mientras el incremento de aprendizaje sea superior a dicho valor
- ### MAX_NUM_EPISONES : número máximo de iteraciones que estamos dispuestos a realizar
- ### STEPS_PER_EPISODE: número máximo de pasos a realizar en cada episodio
- ### ALPHA: ratio de aprendizaje del agente 
- ### GAMMA: factor de descuento del agente
- ### NUM_DISCRETE_BINS: número de divisiones en el caso de discretizar el espacio de estados continuo. 


In [9]:
MAX_NUM_EPISODES = 50000
STEPS_PER_EPISODE = 200
EPSILON_MIN = 0.005
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
EPSILON_DECAY = 500 * EPSILON_MIN / max_num_steps
ALPHA = 0.05
GAMMA = 0.98
NUM_DISCRETE_BINS = 30


## QLearner Class
- ### __init__(self, environment)
- ### discretize(self, obs) [-2,2] -> [-2,-1], [-1,0], [0,1], [1,2]
- ### get_action(self, obs)
- ### learn(self, obs, action, reward, next_obs)

In [10]:
class QLearner(object):
    def __init__(self, environment):
        self.obs_shape = environment.observation_space.shape
        self.obs_high = environment.observation_space.high
        self.obs_low = environment.observation_space.low
        self.obs_bins = NUM_DISCRETE_BINS
        self.bin_width = (self.obs_high-self.obs_low)/self.obs_bins
        
        self.action_shape = environment.action_space.n
        self.Q = np.zeros((self.obs_bins+1, self.obs_bins+1, self.action_shape)) #matriz de 31 x 31 x 3
        self.alpha = ALPHA
        self.gamma = GAMMA
        self.epsilon = 1.0
        
    def discretize(self, obs):
        return tuple(((obs-self.obs_low)/self.bin_width).astype(int))
        
    def get_action(self, obs):
        discrete_obs = self.discretize(obs)
        # Selección de la acción en base a Epsilon-Greedy
        if self.epsilon > EPSILON_MIN:
            self.epsilon -= EPSILON_DECAY
        if np.random.random() > self.epsilon: #Con probabilidad 1-epsilon, elegimos la mejor posible
            return np.argmax(self.Q[discrete_obs])
        else:
            return np.random.choice([a for a in range(self.action_shape)])#Con probabilidad epsilon, elegimos una al azar
        
        
    def learn(self, obs, action, reward, next_obs):
        discrete_obs = self.discretize(obs)
        discrete_next_obs = self.discretize(next_obs)
        self.Q[discrete_obs][action] += self.alpha*(reward + self.gamma * np.max(self.Q[discrete_next_obs]) - self.Q[discrete_obs][action])

## Método para entrenar a nuestro agente

In [11]:
from tqdm import tqdm
def train(agent, environment):
    best_reward = -float('inf')
    for episode in tqdm(range(MAX_NUM_EPISODES)):
        done = False
        obs = environment.reset()
        total_reward = 0.0
        while not done:
            action = agent.get_action(obs)# Acción elegida según la ecuación de Q-LEarning
            next_obs, reward, done, info = environment.step(action)
            agent.learn(obs, action, reward, next_obs)
            obs = next_obs
            total_reward += reward
        if total_reward > best_reward:
            best_reward = total_reward
        print("EPisodio número {} con recompensa: {}, mejor recompensa: {}, epsilon: {}".format(episode, total_reward, best_reward, agent.epsilon))
    
    ## De todas las políticas de entrenamiento que hemos obtenido devolvemos la mejor de todas
    return np.argmax(agent.Q, axis = 2)

## Metodo Testeo

In [12]:
def test(agent, environment, policy):
    done = False
    obs = environment.reset()
    total_reward = 0.0
    while not done:
        action = policy[agent.discretize(obs)] #acción que dictamina la política que hemos entrenado
        next_obs, reward, done, info = environment.step(action)
        obs = next_obs
        total_reward += reward
    return total_reward

# Main

In [19]:
if __name__ == "__main__":
    environment = gym.make("MountainCar-v0")
    agent = QLearner(environment)
    learned_policy = train(agent, environment)
    monitor_path = "./monitor_output"
    environment = gym.wrappers.Monitor(environment, monitor_path, force = True)
    for _ in range(1000):
        test(agent, environment, learned_policy)
    environment.close()

  1%|          | 11/1000 [00:00<00:20, 47.27it/s]

EPisodio número 0 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9975000000000058
EPisodio número 1 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9950000000000117
EPisodio número 2 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9925000000000175
EPisodio número 3 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9900000000000233
EPisodio número 4 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9875000000000291
EPisodio número 5 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.985000000000035
EPisodio número 6 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9825000000000408
EPisodio número 7 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9800000000000466
EPisodio número 8 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9775000000000524
EPisodio número 9 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9750000000000583
EPisodio número 10 con recompensa: -200.0

  2%|▏         | 21/1000 [00:00<00:20, 48.23it/s]

EPisodio número 11 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9700000000000699
EPisodio número 12 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9675000000000757
EPisodio número 13 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9650000000000816
EPisodio número 14 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9625000000000874
EPisodio número 15 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9600000000000932
EPisodio número 16 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.957500000000099
EPisodio número 17 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9550000000001049
EPisodio número 18 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9525000000001107
EPisodio número 19 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9500000000001165
EPisodio número 20 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.9475000000001224
EPisodio número 21 con recompen

  3%|▎         | 31/1000 [00:00<00:20, 47.80it/s]

...

EPisodio número 972 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 973 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 974 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 975 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 976 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 977 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 978 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 979 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 980 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 981 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio n

100%|█████████▉| 995/1000 [00:18<00:00, 55.36it/s]

EPisodio número 984 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 985 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 986 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 987 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 988 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 989 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 990 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 991 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 992 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 993 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio n

100%|██████████| 1000/1000 [00:19<00:00, 52.44it/s]


EPisodio número 996 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 997 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 998 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322
EPisodio número 999 con recompensa: -200.0, mejor recompensa: -200.0, epsilon: 0.00498750000066322


In [20]:
show_videos('monitor_output')